In [ ]:
#библиотеки

import sympy as sp # для символьных вычислений
import numpy as np # для вычислений

In [ ]:
#инициализация глобальных переменных

x, y, z = sp.symbols('x y z')
alpha = sp.Symbol('\u03B1')
beta = sp.Symbol('\u03B2') #длина шага
lmbd = sp.Symbol('\u03BB') #коэффициент дробления
eps = sp.Symbol('\u03B5') #точность приближения

f = 8*x**2 + 8*x*y - 2*x*z + 6*y**2 + 4*y*z + 6*z**2 + 4*x - 14*y - 18*z + 134
eps = 0.03
beta = 0.5
lmbd = 0.5


#начальное приближение
x0 = 0.223
y0 = 0.54
z0 = 0.178

In [ ]:
# функции

'''
функция gradient для вычисления градиента функции в точке (x, y, z)
  вход: функция f, координаты точки [x, y, z]
  выход: вектор градиента в данной точке
'''
def gradient(f, point):
  grad = np.array([
      sp.diff(f, x).subs({x: point[0], y: point[1], z: point[2]}),
      sp.diff(f, y).subs({x: point[0], y: point[1], z: point[2]}),
      sp.diff(f, z).subs({x: point[0], y: point[1], z: point[2]})
  ])
  return grad


'''
функция gradient_descent метода градиентного спуска с дроблением шага
  вход: функция f, параметры beta, lambda, epsilon; начальное приближение x0, y0, z0
  выход: названия метода, количества итераций k, координат x_k, градиента в точке x_k, значения f(x_k), погрешности
'''
def gradient_descent(f, beta, lmbd, eps, x0, y0, z0):
  x_k = [x0, y0, z0]
  k = 0
  while np.max( np.abs( gradient(f, x_k))) >= eps:
    alpha_k = beta
    h_k = - gradient(f, x_k)
    x_k_new = np.array(x_k) + alpha_k * h_k
    while f.subs({x: x_k[0], y: x_k[1], z: x_k[2]}) <= f.subs({x: x_k_new[0], y: x_k_new[1], z: x_k_new[2]}):
      alpha_k *= lmbd
      x_k_new = np.array(x_k) + alpha_k * h_k
    k += 1
    x_k = x_k_new.tolist()

  x_analytical = np.array([-20/23, 67/46, 20/23])
  fault = np.abs(f.subs({x: x_analytical[0], y: x_analytical[1], z: x_analytical[2]}) -
                 f.subs({x: x_k[0], y: x_k[1], z: x_k[2]}))
  print ("Gradient descent\nIteration: ", k,"\nPoint:     ", x_k, "\nGradient:  ", gradient(f, x_k), "\nValue:     ", f.subs({x: x_k[0], y: x_k[1], z: x_k[2]}),
         "\nFault:     ", fault)


'''
функция Hessian_matrix для нахождения матрицы вторых производных
  вход: функция f
  выход: матрица вторых производных
'''
def Hessian_matrix(f):
  A = [[0]*3 for i in range(3)]
  A[0][0] = sp.diff(sp.diff(f, x), x)
  A[0][1] = A[1][0] = sp.diff(sp.diff(f, x), y)
  A[0][2] = A[2][0] = sp.diff(sp.diff(f, x), z)
  A[1][1] = sp.diff(sp.diff(f, y), y)
  A[1][2] = A[2][1] = sp.diff(sp.diff(f, y), z)
  A[2][2] = sp.diff(sp.diff(f, z), z)
  return A


'''
функция min_alpha для нахождения оптимального альфа (для минимизации f)
  вход: функция f, координаты точки x_k (список), координаты h_k (список)
  выход: оптимальное значение альфа
'''
def min_alpha (f, x_k, h_k):
    alpha_k = sp.Symbol('alpha_k')
    f_alpha = f.subs({x: x_k[0] + alpha_k * h_k[0], y: x_k[1] + alpha_k * h_k[1], z: x_k[2] + alpha_k * h_k[2]})

    alpha_arr = sp.solve(sp.diff(f_alpha, alpha_k), alpha_k)
    f_alpha_val = []
    for alpha in alpha_arr:
      f_alpha_val.append(f_alpha.subs(alpha_k, alpha))

    return alpha_arr[f_alpha_val.index(min(f_alpha_val))]


'''
функция conjugate_gradients - метод сопряженных градиентов
  вход: функция f, координаты x0, y0, z0 (начального приближения)
  выход: вывод названия метода, количества итераций k, координат x_k, градиента в точке x_k, значения f(x_k), погрешности
'''
def conjugate_gradients (f, x0, y0, z0):
  k = 0
  x_k = np.array([x0, y0, z0])
  h_k = - gradient(f, x_k)

  while np.max(np.abs(gradient(f, x_k))) >= 10**(-5) :

    x_k_new = x_k + min_alpha(f, x_k, h_k) * h_k

    conj_beta_k = np.dot(Hessian_matrix(f) @ h_k, gradient(f, x_k_new)) / np.dot(Hessian_matrix(f) @ h_k, h_k)
    h_k_new = - gradient(f, x_k_new) + conj_beta_k * h_k

    k += 1
    x_k = x_k_new
    h_k = h_k_new

  x_analytical = np.array([-20/23, 67/46, 20/23])
  fault = np.abs(f.subs({x: x_analytical[0], y: x_analytical[1], z: x_analytical[2]}) -
                 f.subs({x: x_k[0], y: x_k[1], z: x_k[2]}))
  print ("Сonjugate gradients \nIteration: ", k,"\nPoint:     ", x_k, "\nGradient:  ", gradient(f, x_k), "\nValue:     ", f.subs({x: x_k[0], y: x_k[1], z: x_k[2]}),
         "\nFault:     ", fault)

In [ ]:
gradient_descent(f, beta, lmbd, eps, x0, y0, z0)

Gradient descent
Iteration:  14 
Point:      [-0.865877460286087, 1.45297292323237, 0.872279623414215] 
Gradient:   [0.0251847744531712 -0.00222610984335248 0.0110020944722464] 
Value:      114.239195754551 
Fault:      6.53197682396467e-5


In [ ]:
conjugate_gradients (f, x0, y0, z0)

Сonjugate gradients 
Iteration:  3 
Point:      [-0.869565217391304 1.45652173913044 0.869565217391305] 
Gradient:   [6.66133814775094e-15 7.99360577730113e-15 8.88178419700125e-15] 
Value:      114.239130434783 
Fault:      1.42108547152020e-14
